In [1]:
!pip3 install -r requirements.txt


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
%load_ext rpy2.ipython

Во время загрузки - Предупреждения:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 


# Part 1

Write a function named pollutantmean that calculates the mean of a pollutant (sulfate or
nitrate) across a specified list of monitors. The function pollutantmean takes three arguments:
directory, pollutant, and id. Given a vector monitor ID numbers, pollutantmean reads
that monitors’ particulate matter data from the directory specified in the directory argument
and returns the mean of the pollutant across all of the monitors, ignoring any missing values
coded as NA.

In [3]:
import glob
import os
import pandas as pd
import numpy as np

In [20]:
def pollutantmean(dirname: str, pollunt: str, ids: str) -> int:
    indexes = [int(v) for v in ids.split(':')]
    start, stop = (indexes[0], indexes[0]) if len(indexes) == 1 else indexes

    id = list(range(start, stop+1))

    total = 0
    counter = 0
    
    files =[str(i).zfill(3) for i in id]

    for file in files:
            data = pd.read_csv(f'{dirname}/{file}.csv')
            total += data[pollunt].sum()
            counter += data[pollunt].count()
    
    return total/counter


In [21]:
pollutantmean('data/specdata', 'sulfate', "1:10")

4.06412824256036

In [22]:
pollutantmean('data/specdata', "nitrate", "70:72")

1.7060473516949153

In [23]:
pollutantmean('data/specdata', "nitrate", "23")

1.2808333333333333

In [25]:
def complete(dirname, ids):
    if type(ids) is str:
        start, stop = ids.split(':')
        start, stop = int(start), int(stop)
        ns = range(start, stop+1) if start < stop else range(stop, start+1)

    elif type(ids) is int: ns = [ids]

    elif type(ids) is list: ns = ids

    else:
        raise ValueError("Data must be in format 'n', or 'm:n', or '[k, m, n]'")
    
    res = pd.DataFrame(columns=['id', 'nobs'])
    
    for element in ns:
        
        if element < 10: element_str = '00' + str(element)
        if element >= 10 and int(element) < 100: element_str = '0' + str(element)
        if element >= 100: element_str = str(element)
        
        files = glob.glob(f'{dirname}/{element_str}.csv')
        dataset = pd.DataFrame()
        
        for file in files:
            data = pd.read_csv(file).dropna()
            amount = len(data.index)
            list_row = {"id":element, "nobs":amount}
            res.loc[len(res)] = list_row
    
    return res

In [17]:
complete("data/specdata", [2, 4, 8, 10, 12])

,id,nobs
0,2,1041
1,4,474
2,8,192
3,10,148
4,12,96


In [18]:
def corr(dirname, threshold):
    sulf = []
    nitr = []
    for i in range(1, 332+1):
        if i < 10: i_str = '00' + str(i)
        if i >= 10 and int(i) < 100: i_str = '0' + str(i)
        if i >= 100: i_str = str(i)
        files = glob.glob(f'{dirname}/{i_str}.csv')
        dataset = pd.DataFrame()
        for file in files:
            data = pd.read_csv(file).dropna()
            amount = len(data.index)
            if amount >= threshold:
                sulf += data['sulfate'].tolist()
                nitr += data['nitrate'].tolist()
            else: continue
    if len(sulf) == 0 or len(nitr) == 0: return 0
    else: return np.corrcoef(np.array(sulf), np.array(nitr))[0,1] 

In [19]:
corr("data/specdata", 150)

0.06069887784423783